In [ ]:
# from __future__ import print_function

import sys

import numpy as np
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
import pyspark.sql.functions as F

import pytz
dbmap = {"banda_guest": "1"}
# print(__name__)builtins
if __name__ == "builtins":
    spark = SparkSession \
        .builder \
        .appName("Python Demo") \
        .config("hive.metastore.client.factory.class",
                "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
        .config("spark.driver.maxResultSize", "4g") \
        .enableHiveSupport() \
        .getOrCreate()
    spark.conf.set("hive.exec.dynamic.partition.mode", "nonstrict");
    spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
    databases = spark.sql("show databases")
    databases = databases.collect()
    tablelist=["`banda-etl-s3`.`t_channel_details`","`banda-etl-s3`.`t_customer_install_info`","`banda-etl-s3`.`t_customer`"];
    df1=spark.sql("""
        select * 
        from
        (
        SELECT date(cd.create_time + interval '7' hour) as date,'Total' as mark,
        count(distinct cd.id) as install_count,
        count(if(date(cu.create_time + interval '7' hour) >= date(cii.install_time + interval '7' hour),cii.customer_id,null)) as register_user,

        count(if(date(cii.install_time + interval '7' hour) = date(cu.create_time + interval '7' hour),cii.customer_id,null)) as T0_count,

            count(if(date(cii.install_time + interval '7' hour + interval '1' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T1_count,

            count(if(date(cii.install_time + interval '7' hour + interval '2' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T2_count,

            count(if(date(cii.install_time + interval '7' hour + interval '3' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T3_count,

            count(if(date(cii.install_time + interval '7' hour + interval '4' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T4_count,

            count(if(date(cii.install_time + interval '7' hour + interval '5' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T5_count,

            count(if(date(cii.install_time + interval '7' hour + interval '7' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T7_count,

            count(if(date(cii.install_time + interval '7' hour + interval '15' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T15_count,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour) = date(cu.create_time + interval '7' hour),cii.customer_id,null)))/(count(distinct cd.id)),4) else 0 end as T0_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '1' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T1_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '2' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T2_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '3' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T3_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '4' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T4_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '5' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T5_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '7' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T7_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '15' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T15_rate,

            case when count(distinct cd.id) > 0 then round(1.0*count(if(date(cu.create_time + interval '7' hour) >= date(cii.install_time + interval '7' hour),cii.customer_id,null))/(count(distinct cd.id)),4) else 0 end as till_now_rate


        FROM `banda-etl-s3`.`t_channel_details` cd
        left join `banda-etl-s3`.`t_customer_install_info` cii
        on cd.id = cii.channel_details_id
        left join `banda-etl-s3`.`t_customer` cu
        on cii.customer_id = cu.id
        where date(cd.create_time + interval '7' hour) >= date '2019-08-01'
        group by date(cd.create_time + interval '7' hour)
        union all

        SELECT date(cd.create_time + interval '7' hour) as date,'Organic' as mark,
        count(distinct cd.id) as install_count,
        count(if(date(cu.create_time + interval '7' hour) >= date(cii.install_time + interval '7' hour),cii.customer_id,null)) as payday_apply_user,

        count(if(date(cii.install_time + interval '7' hour) = date(cu.create_time + interval '7' hour),cii.customer_id,null)) as T0_count,

            count(if(date(cii.install_time + interval '7' hour + interval '1' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T1_count,

            count(if(date(cii.install_time + interval '7' hour + interval '2' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T2_count,

            count(if(date(cii.install_time + interval '7' hour + interval '3' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T3_count,

            count(if(date(cii.install_time + interval '7' hour + interval '4' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T4_count,

            count(if(date(cii.install_time + interval '7' hour + interval '5' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T5_count,

            count(if(date(cii.install_time + interval '7' hour + interval '7' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T7_count,

            count(if(date(cii.install_time + interval '7' hour + interval '15' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T15_count,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour) = date(cu.create_time + interval '7' hour),cii.customer_id,null)))/(count(distinct cd.id)),4) else 0 end as T0_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '1' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T1_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '2' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T2_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '3' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T3_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '4' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T4_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '5' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T5_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '7' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T7_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '15' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T15_rate,

            case when count(distinct cd.id) > 0 then round(1.0*count(if(date(cu.create_time + interval '7' hour) >= date(cii.install_time + interval '7' hour),cii.customer_id,null))/(count(distinct cd.id)),4) else 0 end as till_now_rate


        FROM `banda-etl-s3`.`t_channel_details` cd
        left join `banda-etl-s3`.`t_customer_install_info` cii
        on cd.id = cii.channel_details_id
        left join `banda-etl-s3`.`t_customer` cu
        on cii.customer_id = cu.id
        where date(cd.create_time + interval '7' hour) >= date '2019-08-01' and cd.media_source = 'Organic'
        group by date(cd.create_time + interval '7' hour)
        union all
        SELECT date(cd.create_time + interval '7' hour) as date,'Others' as mark,
        count(distinct cd.id) as install_count,
        count(if(date(cu.create_time + interval '7' hour) >= date(cii.install_time + interval '7' hour),cii.customer_id,null)) as payday_apply_user,

        count(if(date(cii.install_time + interval '7' hour) = date(cu.create_time + interval '7' hour),cii.customer_id,null)) as T0_count,

            count(if(date(cii.install_time + interval '7' hour + interval '1' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T1_count,

            count(if(date(cii.install_time + interval '7' hour + interval '2' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T2_count,

            count(if(date(cii.install_time + interval '7' hour + interval '3' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T3_count,

            count(if(date(cii.install_time + interval '7' hour + interval '4' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T4_count,

            count(if(date(cii.install_time + interval '7' hour + interval '5' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T5_count,

            count(if(date(cii.install_time + interval '7' hour + interval '7' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T7_count,

            count(if(date(cii.install_time + interval '7' hour + interval '15' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null))  as T15_count,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour) = date(cu.create_time + interval '7' hour),cii.customer_id,null)))/(count(distinct cd.id)),4) else 0 end as T0_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '1' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T1_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '2' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T2_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '3' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T3_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '4' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T4_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '5' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T5_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '7' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T7_rate,

            case when count(distinct cd.id) > 0 then round(1.0*(count(if(date(cii.install_time + interval '7' hour + interval '15' day) >= date(cu.create_time + interval '7' hour),cii.customer_id,null)) )/(count(distinct cd.id)),4) else 0 end as T15_rate,

            case when count(distinct cd.id) > 0 then round(1.0*count(if(date(cu.create_time + interval '7' hour) >= date(cii.install_time + interval '7' hour),cii.customer_id,null))/(count(distinct cd.id)),4) else 0 end as till_now_rate


        FROM 
        (
          select * 
              from `banda-etl-s3`.`t_channel_details` 
              where id  in ( select id from `banda-etl-s3`.`t_channel_details` where ifnull(media_source,'') <> 'Organic')
        ) cd
        left join `banda-etl-s3`.`t_customer_install_info` cii
        on cd.id = cii.channel_details_id
        left join `banda-etl-s3`.`t_customer` cu
        on cii.customer_id = cu.id
        where date(cd.create_time + interval '7' hour) >= date '2019-08-01'
        group by date(cd.create_time + interval '7' hour)


        )
        order by date desc,mark
    """)
    df1.write.mode("overwrite").orc("s3://rupiahplus-data-warehouse/etl/banda/market/id_adapundi_daily_pd_install_to_register")
